# Libraries import

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Colab Notebooks"
# %cd /content/gdrive/My Drive/Colab Notebooks

In [2]:
# # uncomment this cell for first run to install the packages.
# !pip install quandl
# !pip install ta 
# !pip install git+https://github.com/ourownstory/neural_prophet.git
# !pip install -U prophet
# !pip install -U statsmodels
# !pip install -U sklearn
# !pip install -U scikit-learn
# !pip install -U pmdarima

In [132]:
import os
import numpy as np
import pandas as pd
import re
from math import sqrt
import quandl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error
# for mean_squared_error set squared = False for RMSE, (y_true, y_pred)
from sklearn.model_selection import train_test_split

# stats models
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARMAResults, ARIMAResults
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.statespace.varmax import VARMAX, VARMAXResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tools.eval_measures import rmse
import statsmodels.graphics.tsaplots as sgt

from pmdarima import auto_arima
from fbprophet import Prophet
import copy
import random
import sys
import itertools
import warnings

warnings.filterwarnings('ignore')

In [133]:
path = './Outputs/'
if(not os.path.isdir(path)):
    os.mkdir(path)

In [134]:
pd.set_option('display.max_columns', None)

In [135]:
quandl.ApiConfig.api_key = "2At3atzXCrMx_BXfkyz7"
# 6b7jcHuxwW7Wot1kyFNS

# Data to be used

In [136]:
metadata = pd.read_csv('BSE_metadata.csv')
metadata.head()

,code,name,description,refreshed_at,from_date,to_date
0,BOM500002,ABB India Limited EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,3/2/1992,1/18/2021
1,BOM500003,AEGIS LOGISTICS LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,3/2/1992,1/18/2021
2,BOM500008,AMARA RAJA BATTERIES LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,3/2/1992,1/18/2021
3,BOM500009,AMBALAL SARABHAI ENTERPRISES LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,3/2/1992,1/18/2021
4,BOM500010,HOUSING DEVELOPMENT FINANCE CORP.LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,1/3/1992,1/18/2021


## Utility variables and functions

In [137]:
### Codes of selected stocks
healthcare = ['BOM524715','BOM532488','BOM500124']
banking = ['BOM500180','BOM532174','BOM500247']
metals = ['BOM500470','BOM500228','BOM500440']
auto = ['BOM532977','BOM500570','BOM532500']
it = ['BOM500209','BOM532540','BOM507685']
BSE_HEALTHCARE = 'SI0800'
BSE_Information_Technology = 'SI1000'
BSE_BANKEX = 'SIBANK'
BSE_METAL = 'SI1200'
BSE_AUTO = 'SI1900'
SENSEX = 'SENSEX'

stocks = healthcare+banking+metals + auto+it
indices = [BSE_HEALTHCARE, BSE_Information_Technology, BSE_BANKEX, BSE_METAL, BSE_AUTO, SENSEX]
allSymbols = stocks + indices
ScriptWiseData = {}

In [138]:
# Global Variables
RSIColumn = 'RSI'
ATRColumns = ['TR', 'ATR']
MACDColumns = ['FastAverage', 'SlowAverage','MACD','Signal']
BollBandColumns = ['BB_up',"BB_dn","MA","BB_width"]
ADXColumns = ['pos_directional_indicator', 'neg_directional_indicator', 'adx']
OnBalanceVolume = 'OBV'
TechnicalIndicators = ['RSI', 'ATR', 'MACD', 'adx', 'OBV'] #exogenous variables

AllIndicatorColumns = [RSIColumn]+ATRColumns+MACDColumns+BollBandColumns+ADXColumns+[OnBalanceVolume]

In [139]:
start_date = '2020-03-02'
end_date = '2021-03-23'

In [140]:
# get null values of all the stocks
def getNullValuesForSetOfStocks(stockCodeList = allSymbols):
    nullCounts = pd.DataFrame(ScriptWiseData[stockCodeList[0]].isnull().sum(), columns= [getStockNameFromStockCode(stockCodeList[0])])
    for symbol in stockCodeList[1:]:
        currNull = pd.DataFrame(ScriptWiseData[symbol].isnull().sum(), columns= [getStockNameFromStockCode(symbol)])
        nullCounts = nullCounts.merge(currNull, left_index=True, right_index = True, how='outer')
    return nullCounts

In [141]:
def adf_test(series, title = ''):
    """
    Pass in a time series and an optional title
    """
    result = sts.adfuller(series.dropna(), autolag = 'AIC')
    label = ['ADF test statistic', 'p-value', '# lags used', '#observations']
#     print(result)
    out = pd.Series(result[0:4], index=label)
    out['Stock Name'] = getStockNameFromStockCode(title)
    out['Stock Code'] = title
    for key, val in result[4].items():
        out[f'critical value ({key})'] = val
        
#     print(out)
    if(result[1]<0.05):
        return out, 'Stationary'
    else:
        return out, 'Non-Stationary'

In [142]:
stockMetadata = metadata[metadata['code'].isin(allSymbols)]
stockMetadata.head()

,code,name,description,refreshed_at,from_date,to_date
70,BOM500124,DR.REDDY'S LABORATORIES LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,3/2/1992,1/18/2021
104,BOM500180,HDFC Bank Ltd EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/26/1995,1/18/2021
121,BOM500209,INFOSYS LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/2/1995,1/18/2021
133,BOM500228,JSW STEEL LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/2/1995,1/18/2021
146,BOM500247,KOTAK MAHINDRA BANK LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/2/1995,1/18/2021


In [143]:
print(stockMetadata.shape)
print(stockMetadata[stockMetadata['code'].isin(stocks)].shape)
print(stockMetadata[stockMetadata['code'].isin(indices)].shape)

(21, 6)
(15, 6)
(6, 6)


## Map Index to  stocks

In [144]:
stockMetadata.loc[:, 'isStock'] = 0
stockMetadata.loc[:, 'Index'] = ''

stockMetadata.loc[stockMetadata['code'].isin(stocks), 'isStock'] = 1
stockMetadata.reset_index(inplace=True)

In [145]:
stockMetadata.loc[stockMetadata['code'].isin(healthcare), 'Index'] = BSE_HEALTHCARE
stockMetadata.loc[stockMetadata['code'].isin(banking), 'Index'] = BSE_BANKEX
stockMetadata.loc[stockMetadata['code'].isin(metals), 'Index'] = BSE_METAL
stockMetadata.loc[stockMetadata['code'].isin(auto), 'Index'] = BSE_AUTO
stockMetadata.loc[stockMetadata['code'].isin(it), 'Index'] = BSE_Information_Technology

In [146]:
stockMetadata.head()

,index,code,name,description,refreshed_at,from_date,to_date,isStock,Index
0,70,BOM500124,DR.REDDY'S LABORATORIES LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,3/2/1992,1/18/2021,1,SI0800
1,104,BOM500180,HDFC Bank Ltd EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/26/1995,1/18/2021,1,SIBANK
2,121,BOM500209,INFOSYS LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/2/1995,1/18/2021,1,SI1000
3,133,BOM500228,JSW STEEL LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/2/1995,1/18/2021,1,SI1200
4,146,BOM500247,KOTAK MAHINDRA BANK LTD. EOD Prices,End of Day prices and additional trading infor...,1/19/2021 14:15,5/2/1995,1/18/2021,1,SIBANK


In [147]:
stockMetadata['isStock'].value_counts()

1    15
0     6
Name: isStock, dtype: int64

# Code to download the data from quandl. 

In [148]:
# # download the required data - needed only 1 time
# i=0
# code = f'BSE/{stockMetadata.loc[i, "code"]}'
# dataset = quandl.get(code, start_date='2000-01-01', end_date='2021-03-23').sort_values(by='Date', ascending=True)
# dataset['StockName']=stockMetadata.loc[i, 'name']
# dataset['code'] = stockMetadata.loc[i, "code"]
# dataset['isStock'] = stockMetadata.loc[i, "isStock"]
# dataset['Index'] = stockMetadata.loc[i, "Index"]
# for i in range(1, len(stockMetadata)):
#     code = f'BSE/{stockMetadata.loc[i, "code"]}'
#     temp = quandl.get(code, start_date='2000-01-01', end_date='2021-03-23').sort_values(by='Date', ascending=True)
#     temp['StockName']=stockMetadata.loc[i, 'name']
#     temp['code'] = stockMetadata.loc[i, "code"]
#     temp['isStock'] = stockMetadata.loc[i, "isStock"]
#     temp['Index'] = stockMetadata.loc[i, "Index"]
# #     print( stockMetadata.loc[i, "Index"])
#     dataset = dataset.append(temp)
# dataset.to_csv('data.csv')
# dataset.head()

In [149]:
dataset = pd.read_csv('data.csv')
dataset.head()

,Date,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O,StockName,code,isStock,Index
0,2000-01-03,1500.0,1557.35,1500.00,1557.35,1544.195154,82330.0,945.0,127133587.0,NaN,NaN,57.35,57.35,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
1,2000-01-04,1601.0,1681.00,1527.00,1647.25,1621.040930,210676.0,3308.0,341514419.0,NaN,NaN,154.00,46.25,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
2,2000-01-05,1600.0,1648.00,1515.50,1559.00,1590.903923,205669.0,2707.0,327199619.0,NaN,NaN,132.50,-41.00,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
3,2000-01-06,1582.0,1683.70,1560.00,1683.70,1623.244182,161085.0,2559.0,261480289.0,NaN,NaN,123.70,101.70,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
4,2000-01-07,1718.0,1718.00,1549.05,1549.05,1593.853264,120952.0,1955.0,192779740.0,NaN,NaN,168.95,-168.95,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800


In [150]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105449 entries, 0 to 105448
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       105449 non-null  object 
 1   Open                       95130 non-null   float64
 2   High                       95130 non-null   float64
 3   Low                        95130 non-null   float64
 4   Close                      105449 non-null  float64
 5   WAP                        74255 non-null   float64
 6   No. of Shares              74255 non-null   float64
 7   No. of Trades              74255 non-null   float64
 8   Total Turnover             74255 non-null   float64
 9   Deliverable Quantity       68110 non-null   float64
 10  % Deli. Qty to Traded Qty  68110 non-null   float64
 11  Spread H-L                 74255 non-null   float64
 12  Spread C-O                 74255 non-null   float64
 13  StockName                  10

### Verify the min and max of dates of all selected stocks and indices

In [151]:
dateRanges = dataset.groupby(['code', 'StockName']).agg({'Date': ['min', 'max']}).reset_index()
dateRanges.columns = ['Stock Code', 'Stock Name', 'First Date', 'Last Date']
dateRanges

,Stock Code,Stock Name,First Date,Last Date
0,BOM500124,DR.REDDY'S LABORATORIES LTD. EOD Prices,2000-01-03,2021-03-23
1,BOM500180,HDFC Bank Ltd EOD Prices,2000-01-03,2021-03-23
2,BOM500209,INFOSYS LTD. EOD Prices,2000-01-03,2021-03-23
3,BOM500228,JSW STEEL LTD. EOD Prices,2000-01-03,2021-03-23
4,BOM500247,KOTAK MAHINDRA BANK LTD. EOD Prices,2000-01-03,2021-03-23
5,BOM500440,HINDALCO INDUSTRIES LTD. EOD Prices,2000-01-03,2021-03-23
6,BOM500470,TATA STEEL LTD. EOD Prices,2000-01-03,2021-03-23
7,BOM500570,TATA MOTORS LTD. EOD Prices,2000-01-03,2021-03-23
8,BOM507685,WIPRO LTD. EOD Prices,2000-01-03,2021-03-23
9,BOM524715,SUN PHARMACEUTICAL INDUSTRIES LTD. EOD Prices,2000-01-03,2021-03-23


In [152]:
# use above ranges to get stockName
def getStockNameFromStockCode(stockCode):
    return dateRanges.loc[dateRanges['Stock Code']==stockCode, 'Stock Name'].values[0].replace('EOD Prices', '');

In [153]:
data = dataset.set_index('Date')
data.head()

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O,StockName,code,isStock,Index
Date,,,,,,,,,,,,,,,,
2000-01-03,1500.0,1557.35,1500.00,1557.35,1544.195154,82330.0,945.0,127133587.0,NaN,NaN,57.35,57.35,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
2000-01-04,1601.0,1681.00,1527.00,1647.25,1621.040930,210676.0,3308.0,341514419.0,NaN,NaN,154.00,46.25,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
2000-01-05,1600.0,1648.00,1515.50,1559.00,1590.903923,205669.0,2707.0,327199619.0,NaN,NaN,132.50,-41.00,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
2000-01-06,1582.0,1683.70,1560.00,1683.70,1623.244182,161085.0,2559.0,261480289.0,NaN,NaN,123.70,101.70,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800
2000-01-07,1718.0,1718.00,1549.05,1549.05,1593.853264,120952.0,1955.0,192779740.0,NaN,NaN,168.95,-168.95,DR.REDDY'S LABORATORIES LTD. EOD Prices,BOM500124,1,SI0800


### Populate dictionary to hold individual Stock Data

In [154]:
# ScriptWiseData = {'Symbol1': <DatqaFrame>, 'Suymbol 2':Dataframe2}
for symbol in allSymbols:
    ScriptWiseData[symbol] = data.loc[data.code==symbol]

### View null values in the data 

In [155]:
getNullValuesForSetOfStocks()

,SUN PHARMACEUTICAL INDUSTRIES LTD.,DIVI'S LABORATORIES LTD.,DR.REDDY'S LABORATORIES LTD.,HDFC Bank Ltd,ICICI BANK LTD.,KOTAK MAHINDRA BANK LTD.,TATA STEEL LTD.,JSW STEEL LTD.,HINDALCO INDUSTRIES LTD.,BAJAJ AUTO LTD.,TATA MOTORS LTD.,MARUTI SUZUKI INDIA LTD.,INFOSYS LTD.,TATA CONSULTANCY SERVICES LTD.,WIPRO LTD.,BSE Healthcare,BSE Information Technology,BSE BANKEX,BSE METAL,BSE AUTO,BSE SENSEX
Open,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3810,3810,369,1165,1165,0
High,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3810,3810,369,1165,1165,0
Low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3810,3810,369,1165,1165,0
Close,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WAP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
No. of Shares,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
No. of Trades,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
Total Turnover,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
Deliverable Quantity,548,57,538,538,544,537,545,580,555,1,542,81,539,2,538,5282,5282,4784,5282,5282,5282
% Deli. Qty to Traded Qty,548,57,538,538,544,537,545,580,555,1,542,81,539,2,538,5282,5282,4784,5282,5282,5282


# Deriving Technical indicators 

### MACD

In [156]:
# function to create data MACD signal and data. 
def MACD(df, span1 = 12, span2= 26, span3 = 9):
    stockData = df.copy()
    stockData['FastAverage'] = stockData['Close'].ewm(span = span1, min_periods = span1).mean()
    stockData['SlowAverage'] = stockData['Close'].ewm(span = span2, min_periods = span2).mean()
    stockData['MACD'] = stockData['FastAverage'] - stockData['SlowAverage']
    stockData['Signal'] = stockData['MACD'].ewm(span=span3, min_periods = span3).mean()
    return stockData[MACDColumns]

In [157]:
# create stock code based dataframes with MACD signal
for symbol in allSymbols:
    macd = MACD(ScriptWiseData[symbol])
    ScriptWiseData[symbol] = ScriptWiseData[symbol].merge(macd, left_index=True, right_index = True, how='left')

### RSI - Relative Strength Index

In [158]:
# Calculating RSI without using loop
def RSI(DF, n=14):
    df =  DF.copy()
    "function to calculate RSI"
    delta = df['Close'].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    rsi = 100 - 100 / (1+rs);
    return pd.DataFrame(rs).rename(columns={'Close':'RSI'})

In [159]:
#RSI 
for symbol in allSymbols:   
    rsi = RSI(data.loc[data.code==symbol])
    ScriptWiseData[symbol] = ScriptWiseData[symbol].merge(rsi, left_index=True, right_index = True, how='left')

### ATR

In [160]:
def ATR(DF,n=20):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2[ATRColumns]

In [161]:
#ATR 
for symbol in allSymbols:
    ScriptWiseData[symbol][ATRColumns] = ATR(data.loc[data.code==symbol])

### Bollinger band

In [162]:
def BollBnd(DF,n=14):
    "function to calculate Bollinger Band"
    df = DF.copy()
    df["MA"] = df['Close'].rolling(n).mean()
    df["BB_up"] = df["MA"] + 2*df['Close'].rolling(n).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    df["BB_dn"] = df["MA"] - 2*df['Close'].rolling(n).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    df["BB_width"] = df["BB_up"] - df["BB_dn"]
#     df.dropna(inplace=True)
    return df[BollBandColumns]

In [163]:
#Boll band
for symbol in allSymbols:
    ScriptWiseData[symbol][BollBandColumns] = BollBnd(data.loc[data.code==symbol])

### OBV

In [164]:
def OBV(DF):
    """function to calculate On Balance Volume"""
    df = DF.copy()
    df['daily_ret'] = df['Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['No. of Shares'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    return df['obv']

In [165]:
#OBV
for symbol in allSymbols:
    ScriptWiseData[symbol]['OBV'] = OBV(data.loc[data.code==symbol])

### ADX

In [166]:
from ta.trend import ADXIndicator
def ADX(DF):
    df = DF.copy()
    adxI = ADXIndicator(df['High'],df['Low'],df['Close'],14,False)
    df['pos_directional_indicator'] = adxI.adx_pos()
    df['neg_directional_indicator'] = adxI.adx_neg()
    df['adx'] = adxI.adx()
    return df[ADXColumns]

In [167]:
# ADX
for symbol in allSymbols:
    ScriptWiseData[symbol][ADXColumns] = ADX(data.loc[data.code==symbol])

In [168]:
getNullValuesForSetOfStocks()

,SUN PHARMACEUTICAL INDUSTRIES LTD.,DIVI'S LABORATORIES LTD.,DR.REDDY'S LABORATORIES LTD.,HDFC Bank Ltd,ICICI BANK LTD.,KOTAK MAHINDRA BANK LTD.,TATA STEEL LTD.,JSW STEEL LTD.,HINDALCO INDUSTRIES LTD.,BAJAJ AUTO LTD.,TATA MOTORS LTD.,MARUTI SUZUKI INDIA LTD.,INFOSYS LTD.,TATA CONSULTANCY SERVICES LTD.,WIPRO LTD.,BSE Healthcare,BSE Information Technology,BSE BANKEX,BSE METAL,BSE AUTO,BSE SENSEX
Open,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3810,3810,369,1165,1165,0
High,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3810,3810,369,1165,1165,0
Low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3810,3810,369,1165,1165,0
Close,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WAP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
No. of Shares,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
No. of Trades,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
Total Turnover,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282
Deliverable Quantity,548,57,538,538,544,537,545,580,555,1,542,81,539,2,538,5282,5282,4784,5282,5282,5282
% Deli. Qty to Traded Qty,548,57,538,538,544,537,545,580,555,1,542,81,539,2,538,5282,5282,4784,5282,5282,5282


# Removing Extra Columms

In [169]:
OriginalScriptWiseData = ScriptWiseData
for symbol in allSymbols:
    ScriptWiseData[symbol] = ScriptWiseData[symbol][['Close']+TechnicalIndicators]

## Dropping null values

In [170]:
# drop rows from the stock whose technical indicator values are null. 
for stock in stocks:
#     ScriptWiseData[stock] = ScriptWiseData[stock].dropna(subset=AllIndicatorColumns)
    ScriptWiseData[stock] = ScriptWiseData[stock].dropna()    

In [171]:
# ScriptWiseData['BOM500180'][ScriptWiseData['BOM500180']['MACD'].isna()]
getNullValuesForSetOfStocks()

,SUN PHARMACEUTICAL INDUSTRIES LTD.,DIVI'S LABORATORIES LTD.,DR.REDDY'S LABORATORIES LTD.,HDFC Bank Ltd,ICICI BANK LTD.,KOTAK MAHINDRA BANK LTD.,TATA STEEL LTD.,JSW STEEL LTD.,HINDALCO INDUSTRIES LTD.,BAJAJ AUTO LTD.,TATA MOTORS LTD.,MARUTI SUZUKI INDIA LTD.,INFOSYS LTD.,TATA CONSULTANCY SERVICES LTD.,WIPRO LTD.,BSE Healthcare,BSE Information Technology,BSE BANKEX,BSE METAL,BSE AUTO,BSE SENSEX
Close,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RSI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,27,27,27,27,27
ATR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3829,3829,388,1184,1184,20
MACD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,25,25,25,25,25
adx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5255,5255,4757,5255,5255,0
OBV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5282,5282,4784,5282,5282,5282


In [172]:
finalDateRanges =pd.DataFrame(columns = ['Stock Code', 'Stock Name', 'First Date', 'Last Date']) 
for s in stocks:
    r = [s, getStockNameFromStockCode(s), ScriptWiseData[s].index.min(),  ScriptWiseData[s].index.max()]
    finalDateRanges.loc[len(finalDateRanges)] = r
finalDateRanges

,Stock Code,Stock Name,First Date,Last Date
0,BOM524715,SUN PHARMACEUTICAL INDUSTRIES LTD.,2000-02-10,2021-03-23
1,BOM532488,DIVI'S LABORATORIES LTD.,2003-04-23,2021-03-23
2,BOM500124,DR.REDDY'S LABORATORIES LTD.,2000-02-10,2021-03-23
3,BOM500180,HDFC Bank Ltd,2000-02-10,2021-03-23
4,BOM532174,ICICI BANK LTD.,2000-02-10,2021-03-23
5,BOM500247,KOTAK MAHINDRA BANK LTD.,2000-02-10,2021-03-23
6,BOM500470,TATA STEEL LTD.,2000-02-10,2021-03-23
7,BOM500228,JSW STEEL LTD.,2000-02-10,2021-03-23
8,BOM500440,HINDALCO INDUSTRIES LTD.,2000-02-10,2021-03-23
9,BOM532977,BAJAJ AUTO LTD.,2008-07-02,2021-03-23


# ADF Test for closing price

In [173]:
stock = stocks[0]
adf_testResult = pd.DataFrame(adf_test(ScriptWiseData[stocks[0]].Close, stocks[0])[0], columns = ['Value']).transpose().set_index(['Stock Name', 'Stock Code'])
for i in range(1, len(stocks)):
    res = pd.DataFrame(adf_test(ScriptWiseData[stocks[i]].Close, stocks[i])[0], columns = ['Value']).transpose().set_index(['Stock Name', 'Stock Code'])
    adf_testResult = adf_testResult.append(res)
adf_testResult['Result'] = adf_testResult['p-value'].apply(lambda x: 'Stationary' if x<0.05 else 'Non-Stationary')
# adf_testResult.to_csv('Outputs/adfTest.csv')
adf_testResult.head()

,,ADF test statistic,p-value,# lags used,#observations,critical value (1%),critical value (5%),critical value (10%),Result
Stock Name,Stock Code,,,,,,,,
SUN PHARMACEUTICAL INDUSTRIES LTD.,BOM524715,-5.025755,0.00002,0.0,5253.0,-3.431595,-2.86209,-2.567063,Stationary
DIVI'S LABORATORIES LTD.,BOM532488,-3.571373,0.006333,32.0,4424.0,-3.431829,-2.862194,-2.567118,Stationary
DR.REDDY'S LABORATORIES LTD.,BOM500124,-0.081726,0.951229,28.0,5226.0,-3.431602,-2.862093,-2.567064,Non-Stationary
HDFC Bank Ltd,BOM500180,-2.223959,0.197665,0.0,5254.0,-3.431595,-2.86209,-2.567063,Non-Stationary
ICICI BANK LTD.,BOM532174,-2.658944,0.081416,1.0,5252.0,-3.431596,-2.86209,-2.567063,Non-Stationary


# Train test and validation split

In [174]:
def TrainTestSplitByDateRange(df,test_start_date = '2020-03-02',test_end_date = '2021-03-23'):
    train = df.loc[:test_start_date]
    test = df.loc[test_start_date:test_end_date]
    return train, test[1:]

# Code Addition Tavish -----------------------------------------------

#ARIMA 

### AutoARIMA DataFrame


In [175]:
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

In [176]:
AutoArimaResults = pd.DataFrame(columns=['p Closing Values', 'd Closing Values', 'q Closing Values','p N Closing Values', 'd N Closing Values', 'q N Closing Values','p Closing Percentage', 'd Closing Percentage', 'q Closing Percentage'], index = stocks)
#AutoArimaResults = pd.read_csv('Outputs/AutoArimaResults.csv',index_col='Unnamed: 0')


In [179]:
# AutoArimaResults.to_csv('Outputs/AutoArimaResults.csv')

In [180]:
def CalculateAutoArima(stockcode, data, parameter ,p ,d,q):
    if(AutoArimaResults.loc[:, [p, d, q]].isnull().sum(axis=1)[stockcode] > 0):
        params = auto_arima(data[parameter], maxiter = 1000)
        AutoArimaResults.loc[stockcode, [p,d,q]] = params.order
        return params.order
    else:
        return tuple(AutoArimaResults.loc[stockcode, [p, d, q]])

### Common Functions

In [181]:
def errorEvaluation(actual, pred):
    RMSE = mean_squared_error(actual, pred, squared=False)
    MAPE = mean_absolute_percentage_error(actual,pred)
    R2 = r2_score(actual,pred)
    Noise = float(pred.diff().abs().mean())
    return [RMSE, MAPE, R2, Noise]

In [182]:
def modified_prediction_frame(_df_pred, index):
    _df_pred = _df_pred.to_frame()
    _df_pred['Date'] = index
    pd.to_datetime(_df_pred['Date'])
    _df_pred.set_index('Date', inplace=True)
    _df_pred.rename(columns={0 : 'Predicted'}, inplace=True)
    return _df_pred

## ARIMA for Closing Values 

# Tavish - Fix this one too. 

### Generic Function For ARIMA Closing Prices

In [183]:
def ARIMA_Model_Generic(StockCode, data, feature, Results, Order, train_data, test_data):
    print("Modeling for the Stock: {}".format(StockCode))

    start = len(train_data)
    end = len(train_data) + len(test_data) - 1

    print("ADF Test for the given series.")
    adf, Stationarity = adf_test(train_data[feature], title=StockCode)
    print(adf)
    print('----------------------------------------------------------------------------')
    Results.loc[StockCode,'Stationarity'] = Stationarity
    model = ARIMA(train_data[feature], order = Order)

    results = model.fit()
    print(results.params)
    print("Order: " + str(Order) )
    print('---------------------------------------------------------------------------')
    Results.loc[StockCode, 'Order'] = str(Order)

    predictions = results.predict(start = start, end = end, typ ='levels')
    predictions = modified_prediction_frame(predictions, test_data[feature].index)

    errorEval = errorEvaluation(test_data[feature], predictions)
    RMSE, MAPE, R2, Noise = errorEval
    Results.loc[StockCode, ['RMSE', 'MAPE', 'R2', 'Noise']] = errorEval
    print("RMSE: {0} \nMape: {1} \nR2: {2} \nNoise: {3}".format(RMSE, MAPE, R2, Noise))
    print('---------------------------------------------------------------------------')

    print('Description of the Predicted Data')
    print(predictions.describe())
    pred_mean = predictions.mean(numeric_only=True)
    Results.loc[StockCode, 'Predictions Mean'] = str(float(pred_mean))
    Results.loc[StockCode, 'Model'] = model
    print('---------------------------------------------------------------------------')

    print('Description of the Actual Data')
    print(test_data[feature].describe())
    Results.loc[StockCode, 'Actual Mean'] = str(test_data[feature].mean())
    return predictions


In [184]:
try:
    ArimaModelPricesResults = pd.read_csv('./Outputs/ArimaModelPricesResults.csv')
except FileNotFoundError:
    ArimaModelPricesResults = pd.DataFrame(columns=['Stationarity', 'Order', 'RMSE', 'MAPE', 'R2', 'Noise', 'Predictions Mean', 'Actual Mean', 'Model'], index = stocks)

In [186]:
feature = 'Close'
for stock in stocks:
    print("Working for Stock - {}".format(stock))
    try:
        train_data, test_data = TrainTestSplitByDateRange(ScriptWiseData[stock])
        Order = CalculateAutoArima(stock, train_data, feature,  p= 'p Closing Values',d = 'd Closing Values', q = 'q Closing Values')
        ARIMA_Model_Generic(StockCode = stock, data = ScriptWiseData, feature = feature, Results=ArimaModelPricesResults, Order=Order, train_data=train_data, test_data=test_data)
    except Exception as ex:
        pass

Working for Stock - BOM524715
Modeling for the Stock: BOM524715
ADF Test for the given series.
ADF test statistic                                -4.874453
p-value                                            0.000039
# lags used                                             0.0
#observations                                        4989.0
Stock Name              SUN PHARMACEUTICAL INDUSTRIES LTD. 
Stock Code                                        BOM524715
critical value (1%)                               -3.431661
critical value (5%)                                -2.86212
critical value (10%)                              -2.567078
dtype: object
----------------------------------------------------------------------------
const           -0.356600
ma.L1.D.Close    0.048973
dtype: float64
Order: (0, 1, 1)
---------------------------------------------------------------------------
RMSE: 212.88049529842894 
Mape: 0.35827202842248984 
R2: -8.61615470259654 
Noise: 0.3566004015639237
------------

ADF test statistic             -2.627445
p-value                         0.087457
# lags used                          9.0
#observations                     4981.0
Stock Name              TATA STEEL LTD. 
Stock Code                     BOM500470
critical value (1%)            -3.431664
critical value (5%)             -2.86212
critical value (10%)           -2.567079
dtype: object
----------------------------------------------------------------------------
const            0.044890
ar.L1.D.Close    1.303208
ar.L2.D.Close   -0.837643
ma.L1.D.Close   -1.264871
ma.L2.D.Close    0.796480
dtype: float64
Order: (2, 1, 2)
---------------------------------------------------------------------------
RMSE: 174.74806565713232 
Mape: 0.24724915397231928 
R2: -0.22433390957361032 
Noise: 0.07034459453211661
---------------------------------------------------------------------------
Description of the Predicted Data
        Predicted
count  264.000000
mean   370.911658
std      3.442962
min    363.514

Modeling for the Stock: BOM532540
ADF Test for the given series.
ADF test statistic                            -1.981595
p-value                                        0.294701
# lags used                                         0.0
#observations                                    3823.0
Stock Name              TATA CONSULTANCY SERVICES LTD. 
Stock Code                                    BOM532540
critical value (1%)                           -3.432062
critical value (5%)                           -2.862296
critical value (10%)                          -2.567173
dtype: object
----------------------------------------------------------------------------
const            0.249135
ar.L1.D.Close    0.241147
ar.L2.D.Close    0.701390
ma.L1.D.Close   -0.233018
ma.L2.D.Close   -0.734857
dtype: float64
Order: (2, 1, 2)
---------------------------------------------------------------------------
RMSE: 569.6074769752083 
Mape: 0.16728376661836108 
R2: -0.48346918622342616 
Noise: 0.509807839817975

In [187]:
ArimaModelPricesResults.to_csv('./Outputs/ArimaModelPricesResults.csv')

## ARIMA for Normalized Closing Price

### Normalizing the Values  - Normalized_Prices

# TAVISH  - CREATE A COPY HERE

In [188]:
normalized_columns = ['Normalized Closing', 'Normalized RSI', 'Normalized ATR', 'Normalized MACD', 'Normalized ADX', 'Normalized OBV']
normalized_TechnicalIndicators = ['Normalized RSI', 'Normalized ATR', 'Normalized MACD', 'Normalized ADX', 'Normalized OBV']


In [189]:
cols = list(ScriptWiseData[stock].columns)
cols

['Close', 'RSI', 'ATR', 'MACD', 'adx', 'OBV']

In [190]:
# first value normalization
for stock in stocks:
    cols = list(ScriptWiseData[stock].columns)
    for count in range(len(cols)):
        benchmark_prices = ScriptWiseData[stock][cols[count]].iloc[0]
        ScriptWiseData[stock][normalized_columns[count]] = ScriptWiseData[stock][cols[count]].div(benchmark_prices).mul(100)


###  ARIMA Normalized Closing Prices

In [191]:
try:
    ArimaModelPricesResults_Norm = pd.read_csv('./Outputs/ArimaModelPricesResults_Norm.csv')
except FileNotFoundError:
    ArimaModelPricesResults_Norm = pd.DataFrame(columns=['Stationarity', 'Order', 'RMSE', 'MAPE', 'R2','Noise', 'Predictions Mean', 'Actual Mean'], index = stocks)

In [192]:
for stock in stocks:
    print("Working for Stock {}".format(stock))
    train_data, test_data = TrainTestSplitByDateRange(ScriptWiseData[stock])
    feature = 'Normalized Closing'
    Order = CalculateAutoArima(stock, train_data, feature,'p N Closing Values', 'd N Closing Values', 'q N Closing Values')
    ARIMA_Model_Generic(StockCode= stock, data = ScriptWiseData, feature = feature, Results=ArimaModelPricesResults_Norm, Order=Order, train_data=train_data, test_data=test_data)

Working for Stock BOM524715
Modeling for the Stock: BOM524715
ADF Test for the given series.
ADF test statistic                                -4.874453
p-value                                            0.000039
# lags used                                             0.0
#observations                                        4989.0
Stock Name              SUN PHARMACEUTICAL INDUSTRIES LTD. 
Stock Code                                        BOM524715
critical value (1%)                               -3.431661
critical value (5%)                                -2.86212
critical value (10%)                              -2.567078
dtype: object
----------------------------------------------------------------------------
const                        -0.016625
ma.L1.D.Normalized Closing    0.048973
dtype: float64
Order: (0, 1, 1)
---------------------------------------------------------------------------
RMSE: 9.92416207259835 
Mape: 0.35827635698993954 
R2: -8.616399111548606 
Noise: 0.016624

const                         0.168616
ar.L1.D.Normalized Closing   -0.277993
ma.L1.D.Normalized Closing    0.301800
ma.L2.D.Normalized Closing   -0.014911
dtype: float64
Order: (1, 1, 2)
---------------------------------------------------------------------------
RMSE: 170.65112787877953 
Mape: 0.19157833047536177 
R2: -0.025825255823427318 
Noise: 0.16858597314443183
---------------------------------------------------------------------------
Description of the Predicted Data
        Predicted
count  264.000000
mean   963.447649
std     12.874576
min    941.282583
25%    952.361091
50%    963.447625
75%    974.534160
max    985.620694
---------------------------------------------------------------------------
Description of the Actual Data
count     264.000000
mean      898.687886
std       168.809374
min       647.074390
25%       769.112026
50%       811.820053
75%      1085.952661
max      1188.856219
Name: Normalized Closing, dtype: float64
Working for Stock BOM500470
Modeling for 

Modeling for the Stock: BOM532500
ADF Test for the given series.
ADF test statistic                      -0.588969
p-value                                  0.873455
# lags used                                  29.0
#observations                              4082.0
Stock Name              MARUTI SUZUKI INDIA LTD. 
Stock Code                              BOM532500
critical value (1%)                     -3.431953
critical value (5%)                     -2.862248
critical value (10%)                    -2.567147
dtype: object
----------------------------------------------------------------------------
const                         0.679097
ar.L1.D.Normalized Closing   -0.897258
ar.L2.D.Normalized Closing   -0.799629
ma.L1.D.Normalized Closing    0.947836
ma.L2.D.Normalized Closing    0.821221
dtype: float64
Order: (2, 1, 2)
---------------------------------------------------------------------------
RMSE: 406.0167903063617 
Mape: 0.12355597168634051 
R2: 0.18414988603428706 
Noise: 0.73132

In [193]:
ArimaModelPricesResults_Norm.to_csv('./Outputs/ArimaModelPricesResults_Norm.csv')

## ARIMA Model For Returns

### Convert Closing price to percentage change



> revisit normalization for this section - consider normalizing in range -1 and 1 --- Not done yet



In [194]:
for s in stocks:
    ScriptWiseData[s]['Stock_Returns'] = ScriptWiseData[s]['Close'].pct_change(1).mul(100) 
    ScriptWiseData[s] = ScriptWiseData[s][1:] # Because of NAN first value

In [195]:
stock = stocks[0]
adf_testResult = pd.DataFrame(adf_test(ScriptWiseData[stocks[0]].Stock_Returns, stocks[0])[0], columns = ['Value']).transpose().set_index(['Stock Name', 'Stock Code'])
for i in range(1, len(stocks)):
    res = pd.DataFrame(adf_test(ScriptWiseData[stocks[i]].Stock_Returns, stocks[i])[0], columns = ['Value']).transpose().set_index(['Stock Name', 'Stock Code'])
    adf_testResult = adf_testResult.append(res)
adf_testResult['Result'] = adf_testResult['p-value'].apply(lambda x: 'Stationary' if x<0.05 else 'Non-Stationary')
# adf_testResult.to_csv('Outputs/adfTestReturns.csv')
adf_testResult.to_csv('Outputs/adfTestReturns.csv')
adf_testResult.head()

,,ADF test statistic,p-value,# lags used,#observations,critical value (1%),critical value (5%),critical value (10%),Result
Stock Name,Stock Code,,,,,,,,
SUN PHARMACEUTICAL INDUSTRIES LTD.,BOM524715,-51.489152,0.0,1.0,5251.0,-3.431596,-2.862091,-2.567063,Stationary
DIVI'S LABORATORIES LTD.,BOM532488,-13.923122,0.0,23.0,4432.0,-3.431826,-2.862192,-2.567117,Stationary
DR.REDDY'S LABORATORIES LTD.,BOM500124,-71.618449,0.0,0.0,5253.0,-3.431595,-2.86209,-2.567063,Stationary
HDFC Bank Ltd,BOM500180,-54.580752,0.0,1.0,5252.0,-3.431596,-2.86209,-2.567063,Stationary
ICICI BANK LTD.,BOM532174,-32.126798,0.0,5.0,5247.0,-3.431597,-2.862091,-2.567063,Stationary


### ARIMA for percentage Returns

In [196]:
try:
    ArimaModelPricesResults_Returns = pd.read_csv('./Outputs/ArimaModelPricesResults_Returns.csv')
except FileNotFoundError:
    ArimaModelPricesResults_Returns = pd.DataFrame(columns=['Stationarity', 'Order', 'RMSE', 'MAPE', 'R2', 'Noise', 'Predictions Mean', 'Actual Mean'], index = stocks)

In [197]:
for stock in stocks:
    try:
        print("Working for Stock {}".format(stock))
        train_data, test_data = TrainTestSplitByDateRange(ScriptWiseData[stock])
        feature = 'Stock_Returns'
        Order = CalculateAutoArima(stock, train_data, feature, p = 'p Closing Percentage', d = 'd Closing Percentage', q = 'q Closing Percentage')
        ARIMA_Model_Generic(StockCode= stock, data = ScriptWiseData, feature = feature, Results=ArimaModelPricesResults_Returns, Order=Order, train_data=train_data, test_data=test_data)
    except Exception as e:
        print(e)

Working for Stock BOM524715
Modeling for the Stock: BOM524715
ADF Test for the given series.
ADF test statistic                                -50.24951
p-value                                                 0.0
# lags used                                             1.0
#observations                                        4987.0
Stock Name              SUN PHARMACEUTICAL INDUSTRIES LTD. 
Stock Code                                        BOM524715
critical value (1%)                               -3.431662
critical value (5%)                                -2.86212
critical value (10%)                              -2.567079
dtype: object
----------------------------------------------------------------------------
const                  0.026617
ar.L1.Stock_Returns    0.854584
ma.L1.Stock_Returns   -0.823289
ma.L2.Stock_Returns   -0.043835
dtype: float64
Order: (1, 0, 2)
---------------------------------------------------------------------------
RMSE: 2.4181118881512624 
Mape: 1.016610

Modeling for the Stock: BOM500470
ADF Test for the given series.
ADF test statistic            -21.472392
p-value                              0.0
# lags used                          8.0
#observations                     4981.0
Stock Name              TATA STEEL LTD. 
Stock Code                     BOM500470
critical value (1%)            -3.431664
critical value (5%)             -2.86212
critical value (10%)           -2.567079
dtype: object
----------------------------------------------------------------------------
const                  0.057312
ar.L1.Stock_Returns   -0.731096
ar.L2.Stock_Returns    0.163577
ma.L1.Stock_Returns    0.774936
ma.L2.Stock_Returns   -0.131175
dtype: float64
Order: (2, 0, 2)
---------------------------------------------------------------------------
RMSE: 3.110184741793508 
Mape: 1.0106859038342766 
R2: -0.007886769663874027 
Noise: 0.003501333333946175
---------------------------------------------------------------------------
Description of the Predic

Modeling for the Stock: BOM500209
ADF Test for the given series.
ADF test statistic         -42.017994
p-value                           0.0
# lags used                       2.0
#observations                  4987.0
Stock Name              INFOSYS LTD. 
Stock Code                  BOM500209
critical value (1%)         -3.431662
critical value (5%)          -2.86212
critical value (10%)        -2.567079
dtype: object
----------------------------------------------------------------------------
const                  0.006359
ar.L1.Stock_Returns   -0.506842
ar.L2.Stock_Returns   -0.840412
ma.L1.Stock_Returns    0.526018
ma.L2.Stock_Returns    0.839451
dtype: float64
Order: (2, 0, 2)
---------------------------------------------------------------------------
RMSE: 2.505932975127475 
Mape: 0.9944169208686222 
R2: -0.010578621651476938 
Noise: 0.002317779984583498
---------------------------------------------------------------------------
Description of the Predicted Data
        Predicted


In [198]:
ArimaModelPricesResults_Returns

,Unnamed: 0,Unnamed: 0.1,Stationarity,Order,RMSE,MAPE,R2,Noise,Predictions Mean,Actual Mean,Model
0,0.0,BOM524715,Stationary,"(1, 0, 2)",2.418112,1.016611e+00,-0.005038,0.000627,0.030467,0.205739,NaN
1,1.0,BOM532488,Stationary,"(3, 0, 1)",2.371792,1.438822e+00,-0.002710,0.000308,0.106209,0.220249,NaN
2,2.0,BOM500124,Stationary,"(0, 0, 0)",2.254461,1.032354e+00,-0.003719,0.000000,0.043645,0.180882,NaN
3,3.0,BOM500180,Stationary,"(1, 0, 1)",2.651915,1.023123e+00,-0.000856,0.000321,0.071988,0.125669,NaN
4,4.0,BOM532174,Stationary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,BOM500247,Stationary,"(0, 0, 1)",2.914117,1.062708e+00,-0.000089,0.000187,0.10389,0.090105,NaN
6,6.0,BOM500470,Stationary,"(2, 0, 2)",3.110185,1.010686e+00,-0.007887,0.003501,0.056432,0.317305,NaN
7,7.0,BOM500228,Stationary,"(0, 0, 0)",3.116926,1.627840e+00,-0.005721,0.000000,0.523875,0.28878,NaN
8,8.0,BOM500440,Stationary,"(2, 0, 2)",3.682149,9.859136e-01,-0.009147,0.001817,0.027244,0.368711,NaN
9,9.0,BOM532977,Stationary,"(0, 0, 1)",2.434276,1.173898e+00,0.000658,0.000843,0.086117,0.133604,NaN


In [199]:
ArimaModelPricesResults_Returns.to_csv('./Outputs/ArimaModelPricesResults_Returns.csv')

#ARIMAX

### AutoARIMA DataFrame


In [200]:
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

In [201]:
try:
    AutoArimaResultsForArimax = pd.read_csv('./Outputs/AutoArimaResultsForArimax.csv')
except FileNotFoundError:
    AutoArimaResultsForArimax = pd.DataFrame(columns=['p Closing Values', 'd Closing Values', 'q Closing Values','p N Closing Values', 'd N Closing Values', 'q N Closing Values','p Closing Percentage', 'd Closing Percentage', 'q Closing Percentage'], index = stocks)

AutoArimaResultsForArimax

,Unnamed: 0,p Closing Values,d Closing Values,q Closing Values,p N Closing Values,d N Closing Values,q N Closing Values,p Closing Percentage,d Closing Percentage,q Closing Percentage
0,BOM524715,0.0,1.0,1.0,NaN,NaN,NaN,1,0,2
1,BOM532488,NaN,NaN,NaN,NaN,NaN,NaN,3,0,1
2,BOM500124,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,BOM500180,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1
4,BOM532174,NaN,NaN,NaN,NaN,NaN,NaN,2,0,4
5,BOM500247,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
6,BOM500470,NaN,NaN,NaN,NaN,NaN,NaN,2,0,2
7,BOM500228,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
8,BOM500440,NaN,NaN,NaN,NaN,NaN,NaN,2,0,2
9,BOM532977,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1


In [202]:
def CalculateAutoArimaForArimax(stockcode, data, parameter ,p ,d,q):
    if(AutoArimaResultsForArimax.loc[:, [p, d, q]].isnull().sum(axis=1)[stockcode] > 0):
        params = auto_arima(data[parameter], exog = data[TechnicalIndicators], maxiter = 1000)
        AutoArimaResultsForArimax.loc[stockcode, [p,d,q]] = params.order
        return params.order
    else:
        return tuple(AutoArimaResultsForArimax.loc[stockcode, [p, d, q]])

##Generic Function for ARIMAX

# TAVISH - FIX THIS SHIT - EXOG VARIABLE IS INCORRECT

In [203]:
def ARIMAX_Model_Generic(StockCode, data, feature, Results, Order, train_data, test_data, extra_indicators):
    print("Modeling for the Stock: {}".format(StockCode))

    start = len(train_data)
    end = len(train_data) + len(test_data) - 1

    print("ADF Test for the given series.")
    adf, Stationarity = adf_test(train_data[feature], title=StockCode)
    print(adf)
    print('----------------------------------------------------------------------------')
    Results.loc[StockCode,'Stationarity'] = Stationarity

    exog_train = train_data[extra_indicators]
    model = ARIMA(endog = train_data[feature], exog = exog_train , order = Order)
    results = model.fit()
    print(results.params)
    print("Order: " + str(Order) )
    print('---------------------------------------------------------------------------')
    Results.loc[StockCode, 'Order'] = str(Order)

    exog_test = test_data[extra_indicators]
    predictions = results.predict(start = start, end = end, typ ='levels', exog=exog_test)
    predictions = modified_prediction_frame(predictions, test_data[feature].index)

    predictions.plot(figsize=(12,8), legend=True)
    test_data[feature].plot(figsize = (12,8), legend=True)

    errorEval = errorEvaluation(test_data[feature], predictions)
    RMSE, MAPE, R2, Noise = errorEval
    Results.loc[StockCode, ['RMSE', 'MAPE', 'R2', 'Noise']] = errorEval

    print("RMSE: {0} \nMape: {1} \nR2: {2} \nNoise: {3}".format(RMSE, MAPE, R2, Noise))
    print('---------------------------------------------------------------------------')

    print('Description of the Predicted Data')
    print(predictions.describe())
    pred_mean = predictions.mean(numeric_only=True)
    Results.loc[StockCode, 'Predictions Mean'] = str(float(pred_mean))
    print('---------------------------------------------------------------------------')
    
    Results.loc[StockCode, 'Model'] = model
    
    print('Description of the Actual Data')
    print(test_data[feature].describe())
    Results.loc[StockCode, 'Actual Mean'] = str(test_data[feature].mean())
    return predictions

## ARIMAX for Closing Prices

In [204]:
try:
    ArimaxModelPricesResults = pd.read_csv('./Outputs/ArimaModelPricesResults.csv')
except FileNotFoundError:
    ArimaxModelPricesResults = pd.DataFrame(columns=['Stationarity', 'Order', 'RMSE', 'MAPE', 'R2', 'Noise', 'Predictions Mean', 'Actual Mean'], index = stocks)

In [205]:
for stock in stocks:
    print("Working for Stock {}".format(stock))
    train_data, test_data = TrainTestSplitByDateRange(ScriptWiseData[stock])
    feature = 'Close'
    Order = CalculateAutoArimaForArimax(stock, train_data, feature,  p= 'p Closing Values',d = 'd Closing Values', q = 'q Closing Values')
    ARIMAX_Model_Generic(StockCode = stock, data = ScriptWiseData, feature = feature, Results=ArimaxModelPricesResults, Order=Order, train_data=train_data, test_data=test_data,extra_indicators = TechnicalIndicators )

Working for Stock BOM524715


KeyError: 'BOM524715'

### Please check this

In [ ]:
ArimaModelPricesResults.to_csv('./Outputs/ArimaModelPricesResults.csv')

In [ ]:
ArimaModelPricesResults

## ARIMAX Normalized Closing Prices

In [ ]:
try:
    ArimaxModelPricesResults_Norm = pd.read_csv('./Outputs/ArimaxModelPricesResults_Norm.csv')
except FileNotFoundError:
    ArimaxModelPricesResults_Norm = pd.DataFrame(columns=['Stationarity', 'Order', 'RMSE', 'MAPE', 'R2','Noise', 'Predictions Mean', 'Actual Mean'], index = stocks)

In [ ]:
for stock in stocks:
    print("Working for Stock {}".format(stock))
    train_data, test_data = TrainTestSplitByDateRange(ScriptWiseData[stock])
    feature = 'Normalized Closing'
    Order = CalculateAutoArimaForArimax(stock, train_data, feature,'p N Closing Values', 'd N Closing Values', 'q N Closing Values')
    ARIMAX_Model_Generic(StockCode= stock, data = ScriptWiseData, feature = feature, Results=ArimaxModelPricesResults_Norm, Order=Order, train_data=train_data, test_data=test_data,extra_indicators = normalized_TechnicalIndicators)

In [ ]:
ArimaxModelPricesResults_Norm.to_csv('./Outputs/ArimaxModelPricesResults_Norm.csv')

In [ ]:
ArimaxModelPricesResults_Norm

## ARIMAX Returns

In [ ]:
try:
    ArimaxModelPricesResults_Returns = pd.read_csv('./Outputs/ArimaxModelPricesResults_Returns.csv')
except FileNotFoundError:
    ArimaxModelPricesResults_Returns = pd.DataFrame(columns=['Stationarity', 'Order', 'RMSE', 'MAPE', 'R2', 'Noise', 'Predictions Mean', 'Actual Mean'], index = stocks)
    


In [ ]:
for stock in stocks:
    try:
        print("Working for Stock {}".format(stock))
        train_data, test_data = TrainTestSplitByDateRange(ScriptWiseData[stock])
        feature = 'Stock_Returns'
        Order = CalculateAutoArimaForArimax(stock, train_data, feature, p = 'p Closing Percentage', d = 'd Closing Percentage', q = 'q Closing Percentage')
#         ARIMAX_Model_Generic(StockCode= stock, data = ScriptWiseData, feature = feature, Results=ArimaxModelPricesResults_Returns, Order=Order, train_data=train_data, test_data=test_data, extra_indicators = TechnicalIndicators)
    except Exception as e:
        print(e)  

In [ ]:
ArimaxModelPricesResults_Returns.to_csv('./Outputs/ArimaxModelPricesResults_Returns.csv')

In [ ]:
AutoArimaResults.to_csv('./Outputs/AutoArimaResultsForArimax.csv')

# SARIMAX MODEL

In [83]:
def SARIMAXProcedure(stockCode):
    # decompose
    data = ScriptWiseData  [stockCode]
    data.index = pd.to_datetime(data.index)
    data=data.asfreq('b')
    data = data.fillna(method='ffill')
    decompose_result = seasonal_decompose(data['Close'], model='add')
    
    r = auto_arima(data['Close'], exogenous=data[TechnicalIndicators], maxiter=1000) #seasonality, m parameters pending
    p, d, q = r.order # define for P, D, Q, S
        
    train, test = TrainTestSplitByDateRange(data)
    
    model = SARIMAX(train['Close'], exog = train[TechnicalIndicators] ,order = (p, d, q), seasonal_order = (1,1,1,2), enforce_invertibility = False)
    results = model.fit()
    predictions = results.predict(start_date, end_date, exog=test[TechnicalIndicators]).rename('SARIMA')
    
    return decompose_result, results, predictions

In [84]:
decompose_result, model, predictions = SARIMAXProcedure(stocks[2])

### VARMAX MODEL

In [85]:
ScriptWiseData[stocks[0]]

,Close,RSI,ATR,MACD,adx,OBV,Normalized Closing,Normalized RSI,Normalized ATR,Normalized MACD,Normalized ADX,Normalized OBV,Stock_Returns
Date,,,,,,,,,,,,,
2000-02-11,2090.00,1.364001,131.2700,97.046900,19.756105,246481.0,97.431355,81.551223,100.582331,92.376124,98.618868,89.602085,-2.568645
2000-02-14,2125.00,1.489559,128.1250,92.209986,19.499189,267854.0,99.062981,89.058167,98.172554,87.772006,97.336389,97.371712,1.674641
2000-02-15,2068.90,1.225341,126.1700,83.260861,18.662103,248155.0,96.447718,73.261033,96.674584,79.253593,93.157809,90.210627,-2.640000
2000-02-16,2075.00,1.246006,125.6700,75.754360,17.763029,256662.0,96.732087,74.496555,96.291472,72.108373,88.669795,93.303136,0.294843
2000-02-17,2240.95,1.848351,130.8625,81.300839,18.112124,296910.0,104.468323,110.509724,100.270094,77.387905,90.412413,107.934304,7.997590
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-17,585.20,0.682121,17.9600,-1.488529,16.856154,7598758.0,27.280779,40.782849,13.761398,-1.416887,84.142844,2762.340958,-2.798771
2021-03-18,574.50,0.594893,18.3000,-3.999505,18.296442,7374060.0,26.781968,35.567631,14.021914,-3.807013,91.332497,2680.657545,-1.828435
2021-03-19,578.60,0.647393,18.6500,-5.594151,20.001070,7557363.0,26.973101,38.706507,14.290093,-5.324909,99.841688,2747.292827,0.713664


In [86]:
# Train and fit varmax model
def VARMAXProcedure(stockSymbol):
    data = ScriptWiseData[stockSymbol]
    data.index = pd.to_datetime(data.index)
    data=data.asfreq('D')
    data = data.fillna(method='ffill')
    data = data[['Close']+TechnicalIndicators]
    r = auto_arima(data['Close'], exogenous=data[TechnicalIndicators], maxiter=1000) #seasonality, m parameters pending
    p, d, q = r.order
    if(p==0 and d==0):
        p = 1
    # make data stationary based on auto arima output
    
    # train test split  
    train, test = TrainTestSplitByDateRange(data)
    
    # fit the model based on above parameters
    model = VARMAX(train, order=(p,q))
    results = model.fit(maxiter=1000, disp=False)
    forecasts = results.forecast(len(test))

    return results, forecasts, test

In [ ]:
import datetime
try:
    VarmaxResults = pd.read_csv('./Outputs/VarmaxResults.csv')
except FileNotFoundError:
    VarmaxResults = pd.DataFrame(columns = ['StockName','MAE','MAPE', 'RMSE', 'AdjustedR2', 'Noise', 'Model'], index = stocks)
    
for s in stocks:
    print(getStockNameFromStockCode(s))
    try:
        print(datetime.datetime.now())
        if(np.isnan(VarmaxResults.loc[s, 'MAE'])):
            model, forecasts, test = VARMAXProcedure(s)
            print(f'Model Complete for {getStockNameFromStockCode(s)}')
            VarmaxResults.loc[s, ['Model']] =  model
            VarmaxResults.loc[s, ['StockName']] =  getStockNameFromStockCode(s)
            VarmaxResults.loc[s, ['MAE']] =  mean_absolute_error(forecasts.Close, test.Close)
            VarmaxResults.loc[s, ['RMSE']] = mean_squared_error(forecasts.Close, test.Close, squared=False)
            VarmaxResults.loc[s, ['MAPE']] = mean_absolute_percentage_error(forecasts.Close, test.Close)
            VarmaxResults.loc[s, ['AdjustedR2']] = r2_score(forecasts.Close, test.Close)
            VarmaxResults.loc[s, ['Noise']] = forecasts.Close.diff().abs().mean()
            VarmaxResults.to_csv('./Outputs/VarmaxResults.csv')
            print('Success')
        else:
            print('Already exists')
        print(datetime.datetime.now())
    except:
        print(f"Error Occured {getStockNameFromStockCode(s)}", sys.exc_info())

## Prophet models

In [ ]:
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from prophet.utilities import regressor_coefficients
df_start_date = pd.to_datetime(start_date)
df_end_date = pd.to_datetime(end_date)

In [88]:
def ProphetProcedure(stockCode, start_date = start_date, end_date = end_date):
    df = ScriptWiseData[stockCode][['Close']]
    df.index = pd.to_datetime(df.index)
    df = df.reset_index()
    
    df.columns = ['ds', 'y']
    
    test_start_date = pd.to_datetime(start_date)
    test_end_date = pd.to_datetime(end_date)
    
    train = df.loc[df.ds<df_start_date]
    test = df.loc[df.ds>=df_start_date]
    m = Prophet()

    m.fit(train)
    future = m.make_future_dataframe(periods = len(test)-1, freq = "B")
#     test_t = test.set_index('ds')
#     future = future.set_index('ds')
#     future = test_t[].join(future).reset_index()
    forecast = m.predict(future)
    
        # initial
    initial = str(5*365)+' days'

    #period
    period = str(365)+' days'

    #horizon
    horizon = str(365) + ' days'
    df_cv = cross_validation(m, initial = initial, period = period, horizon = horizon)
    
    return m, forecast, df_cv

In [89]:
def ProphetProcedureWithExogenousVariables(stockCode, start_date = start_date, end_date = end_date):
    df = ScriptWiseData[stockCode][['Close']+TechnicalIndicators]
    df.index = pd.to_datetime(df.index)
    df = df.reset_index()
    
    df.columns = ['ds', 'y'] + TechnicalIndicators
    
    test_start_date = pd.to_datetime(start_date)
    test_end_date = pd.to_datetime(end_date)
    
    train = df.loc[df.ds<df_start_date]
    test = df.loc[df.ds>=df_start_date]
#     return df, 1
    m = Prophet()
    for t in TechnicalIndicators:
        m.add_regressor(t)
    m.fit(train)
    future = m.make_future_dataframe(periods = len(test)-1, freq = "B")
    test_t = test.set_index('ds')
    future = future.set_index('ds')
    future = test_t[TechnicalIndicators].join(future).reset_index()
    forecast = m.predict(future)
    
        # initial
    initial = str(5*365)+' days'

    #period
    period = str(365)+' days'

    #horizon
    horizon = str(365) + ' days'
    df_cv = cross_validation(m, initial = initial, period = period, horizon = horizon)
    
    return m, forecast, df_cv

In [90]:
# Hyper parameter tuning
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

# # Generate all combinations of parameters
# all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
# rmses = []  # Store the RMSEs for each params here

# # Use cross validation to evaluate all parameters
# for params in all_params:
#     m = Prophet(**params).fit(df)  # Fit model with given params
#     df_cv = cross_validation(m, cutoffs=cutoffs, horizon='30 days', parallel="processes")
#     df_p = performance_metrics(df_cv, rolling_window=1)
#     rmses.append(df_p['rmse'].values[0])

# # Find the best parameters
# tuning_results = pd.DataFrame(all_params)
# tuning_results['rmse'] = rmses
# print(tuning_results)

In [91]:
try:
    ProphetSingleRegressionResults = pd.read_csv('./Outputs/ProphetSingleRegressionResults.csv')
except FileNotFoundError:
    ProphetSingleRegressionResults = pd.DataFrame(columns = ['StockName','MAE','MAPE', 'RMSE', 'AdjustedR2', 'Noise', 'Model'], index = stocks)
    

In [92]:

# import json
# from prophet.serialize import model_to_json, model_from_json

# with open('serialized_model.json', 'w') as fout:
#     json.dump(model_to_json(m), fout)  # Save model

# with open('serialized_model.json', 'r') as fin:
#     m = model_from_json(json.load(fin))  # Load model

for s in stocks:
    print(s)
    try:
        m, forecast, df_cv = ProphetProcedure(s)
        val_metrics = performance_metrics(df_cv)
        ProphetSingleRegressionResults.loc[s, ['StockName']] =  getStockNameFromStockCode(s)
        ProphetSingleRegressionResults.loc[s, ['MAE']] =  val_metrics['mae'][0]
        ProphetSingleRegressionResults.loc[s, ['RMSE']] = np.sqrt(val_metrics['mse'][0])
        t = df_cv.dropna()
        ProphetSingleRegressionResults.loc[s, ['MAPE']] = mean_absolute_percentage_error(t.y, t.yhat)
        ProphetSingleRegressionResults.loc[s, ['AdjustedR2']] = r2_score(t.y, t.yhat)
        ProphetSingleRegressionResults.loc[s, ['Noise']] = t.yhat.diff().abs().mean()
        ProphetSingleRegressionResults.loc[s, ['Model']] = m
        print('Success')
    except:
        print("Error Occured", sys.exc_info())
        
ProphetSingleRegressionResults.to_csv('./Outputs/ProphetSingleRegressionResults.csv')

BOM524715


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 15 forecasts with cutoffs between 2005-03-03 00:00:00 and 2019-02-28 00:00:00


  0%|          | 0/15 [00:00<?, ?it/s]

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Success
BOM532488


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Success
BOM507685


INFO:fbprophet:Making 15 forecasts with cutoffs between 2005-03-03 00:00:00 and 2019-02-28 00:00:00


  0%|          | 0/15 [00:00<?, ?it/s]

Success


In [93]:
try:
    ProphetMultipleRegressionResults = pd.read_csv('./Outputs/ProphetMultipleRegressionResults.csv')
except FileNotFoundError:
    ProphetMultipleRegressionResults = pd.DataFrame(columns = ['StockName','MAE','MAPE', 'RMSE', 'AdjustedR2', 'Noise', 'Model'], index = stocks)
    

In [94]:
# initalize result variable

for s in stocks:
    print(s)
    # try:
    m, forecast, df_cv = ProphetProcedureWithExogenousVariables(s)
    val_metrics = performance_metrics(df_cv)
    ProphetMultipleRegressionResults.loc[s, ['StockName']] =  getStockNameFromStockCode(s)
    ProphetMultipleRegressionResults.loc[s, ['MAE']] =  val_metrics['mae'][0]
    ProphetMultipleRegressionResults.loc[s, ['RMSE']] = np.sqrt(val_metrics['mse'][0])
    t = df_cv.dropna()
    ProphetMultipleRegressionResults.loc[s, ['MAPE']] = mean_absolute_percentage_error(t.y, t.yhat)
    ProphetMultipleRegressionResults.loc[s, ['AdjustedR2']] = r2_score(t.y, t.yhat)
    ProphetMultipleRegressionResults.loc[s, ['Noise']] = t.yhat.diff().abs().mean()
    ProphetMultipleRegressionResults.loc[s, ['Model']] = m
    print('Success')
    # except:
    #     print("Error Occured", sys.exc_info())
        
ProphetMultipleRegressionResults.to_csv('./Outputs/ProphetMultipleRegressionResults.csv')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


BOM524715


INFO:fbprophet:Making 15 forecasts with cutoffs between 2005-03-03 00:00:00 and 2019-02-28 00:00:00


  0%|          | 0/15 [00:00<?, ?it/s]

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Success
BOM507685


INFO:fbprophet:Making 15 forecasts with cutoffs between 2005-03-03 00:00:00 and 2019-02-28 00:00:00


  0%|          | 0/15 [00:00<?, ?it/s]

Success


In [95]:
# # for plotting
# df = ScriptWiseData[stocks[1]]
# df.index = pd.to_datetime(df.index)
# df = df.reset_index()
# df.columns = ['ds', 'y'] + TechnicalIndicators

ValueError: Length mismatch: Expected axis has 14 elements, new values have 7 elements

In [ ]:
results = performance_metrics(df_cv)
results.head(2)

In [ ]:
m.plot_components(forecast);

In [ ]:
ax = forecast.plot(x='ds',y='yhat',label='Predictions',legend=True,figsize=(12,8))

df.plot(x='ds',y='y',label='Actual Price',legend=True,ax=ax) #,xlim=(df_start_date,df_end_date))

In [ ]:
plot_cross_validation_metric(df_cv, metric='rmse');

## Neural Prophet and AR-Net

In [96]:
from neuralprophet import NeuralProphet, set_random_seed , set_log_level
set_random_seed(0)
set_log_level("ERROR")

In [97]:
df = ScriptWiseData['BOM500180'].reset_index()
df = df[['Date', 'Close']]
df.columns = ['ds', 'y']
df.head()

,ds,y
0,2000-02-11,230.95
1,2000-02-14,234.00
2,2000-02-15,231.00
3,2000-02-16,249.45
4,2000-02-17,250.00


### Basic Model

In [98]:
def BuildNeuralProphetModels(dataset, start_date = start_date, end_date = end_date, n_forecasts = 1, n_lags = 5, columnName = 'Close'):
    # prepare the dataset
    df = dataset.reset_index()
    df = df[['Date', columnName]]
    df.columns = ['ds', 'y']
    
    # train test splilt
    train = df.loc[(df['ds'] < start_date)]
    df_test = df.loc[(df['ds'] >= start_date) & (df['ds'] <= end_date)]
    
    # initalize the model
    m = NeuralProphet(n_forecasts=n_forecasts,n_lags=n_lags)
    
    # split the training set into train and validation set
    df_train, df_val = m.split_df(train, valid_p=0.2, freq='D')

    # fit and test the model
    train_metrics = m.fit(df_train, freq='D', progress_bar = False)
    val_metrics = m.test(df_val)
    
    # make forecasts
    future = m.make_future_dataframe(df_test, periods=len(df_test), n_historic_predictions=len(df_test))
#     future = m.make_future_dataframe(df_test, n_historic_predictions= True)
    forecast = m.predict(future)
    return (m, forecast,  train_metrics, val_metrics)

In [99]:
try:
    NPSingleRegressionResults = pd.read_csv('./Outputs/NeuralProphetSingleRegressionResults.csv')
except FileNotFoundError:
    NPSingleRegressionResults = pd.DataFrame(columns = ['StockName','MAE','MAPE', 'RMSE', 'AdjustedR2', 'Noise', 'Model'], index = stocks)

In [100]:
# initalize result variable
for s in stocks:
    print(s)
    try:
        m, forecast , train_metrics, val_metrics = BuildNeuralProphetModels(ScriptWiseData[s])
        NPSingleRegressionResults.loc[s, ['StockName']] =  getStockNameFromStockCode(s)
        NPSingleRegressionResults.loc[s, ['MAE']] =  val_metrics['MAE'][0]
        NPSingleRegressionResults.loc[s, ['RMSE']] = np.sqrt(val_metrics['MSE'][0])
        t = forecast.dropna()
        NPSingleRegressionResults.loc[s, ['MAPE']] = mean_absolute_percentage_error(t.y, t.yhat1)
        NPSingleRegressionResults.loc[s, ['AdjustedR2']] = r2_score(t.y, t.yhat1)
        NPSingleRegressionResults.loc[s, ['Noise']] = t.yhat1.diff().abs().mean()
        NPSingleRegressionResults.loc[s, ['Model']] = m
        print('Success')
    except:
        print("Error Occured")
        
NPSingleRegressionResults.to_csv('./Outputs/NeuralProphetSingleRegressionResults.csv')

  0%|          | 0/288 [00:00<?, ?it/s]

BOM524715


  0%|          | 0/284 [00:00<?, ?it/s]

Success
BOM532488


  0%|          | 0/288 [00:00<?, ?it/s]

...

  0%|          | 0/288 [00:00<?, ?it/s]

Success
BOM500209
Success
BOM532540


  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

Success
BOM507685
Success


In [101]:
NPSingleRegressionResults

,StockName,MAE,MAPE,RMSE,AdjustedR2,Noise,Model
BOM524715,SUN PHARMACEUTICAL INDUSTRIES LTD.,6.967936,0.015264,9.349832,0.981051,6.441857,<neuralprophet.forecaster.NeuralProphet object...
BOM532488,DIVI'S LABORATORIES LTD.,12.471407,0.012332,19.774289,0.993397,32.323097,<neuralprophet.forecaster.NeuralProphet object...
BOM500124,DR.REDDY'S LABORATORIES LTD.,31.186486,0.011435,40.884414,0.988363,39.957004,<neuralprophet.forecaster.NeuralProphet object...
BOM500180,HDFC Bank Ltd,11.412683,0.015701,31.773432,0.990267,13.985646,<neuralprophet.forecaster.NeuralProphet object...
BOM532174,ICICI BANK LTD.,8.220376,0.04107,10.055294,0.963905,6.708942,<neuralprophet.forecaster.NeuralProphet object...
BOM500247,KOTAK MAHINDRA BANK LTD.,8.2158,0.014112,12.353465,0.988528,20.171779,<neuralprophet.forecaster.NeuralProphet object...
BOM500470,TATA STEEL LTD.,5.487968,0.015938,7.967524,0.995923,6.576811,<neuralprophet.forecaster.NeuralProphet object...
BOM500228,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BOM500440,HINDALCO INDUSTRIES LTD.,2.537102,0.019312,3.552581,0.994652,3.264525,<neuralprophet.forecaster.NeuralProphet object...
BOM532977,BAJAJ AUTO LTD.,25.83194,0.01367,35.852435,0.989586,27.883971,<neuralprophet.forecaster.NeuralProphet object...


#### Plot the forecast

In [102]:
# forecasts_plot = m.plot(forecast)

In [103]:
# fig_comp = m.plot_components(forecast);

In [104]:
# fig_param = m.plot_parameters()

#### Validating the dataset

In [105]:
# m = NeuralProphet()
# df_train, df_val = m.split_df(df, valid_p=0.2, freq='D')

In [106]:
# train_metrics = m.fit(df_train, freq='D')
# val_metrics = m.test(df_val)

## Multiple regressors

In [107]:
def BuildNeuralProphetModelsWithMultipleRegressors(dataset, start_date = start_date, end_date = end_date, n_forecasts = 1, n_lags = 5):
    # prepare the dataset
    df = dataset.reset_index()
    df = df[['Date', 'Close']+TechnicalIndicators]
    df.columns = ['ds', 'y', 'A', 'B', 'C', 'D', 'E']
    
    # train test splilt
    train = df.loc[(df['ds'] < start_date)]
    df_test = df.loc[(df['ds'] >= start_date) & (df['ds'] <= end_date)]
    
    # initalize the model
    m = NeuralProphet(n_forecasts=n_forecasts,n_lags=n_lags)
    # lagged regressors
    m.add_lagged_regressor(name='A')
    m.add_lagged_regressor(name='B')
    m.add_lagged_regressor(name='C')
    m.add_lagged_regressor(name='D')
    m.add_lagged_regressor(name='E')
    
    # split the training set into train and validation set
    df_train, df_val = m.split_df(train, valid_p=0.2, freq='D')

    # fit and test the model
    train_metrics = m.fit(df_train, freq='D',progress_bar = False)
    val_metrics = m.test(df_val)
    
    # make forecasts
    # future = m.make_future_dataframe(df_test, periods=len(df_test), n_historic_predictions=len(df_test))
    future = m.make_future_dataframe(df_test, n_historic_predictions= True)
    forecast = m.predict(future)
    return (m, forecast,  train_metrics, val_metrics)

In [108]:
try:
    NPExogenousRegressionResults = pd.read_csv('./Outputs/NeuralProphetRegressionWithExogenousVariablesResults.csv')
except FileNotFoundError:
    NPExogenousRegressionResults = pd.DataFrame(columns = ['StockName','MAE','MAPE', 'RMSE', 'AdjustedR2', 'Noise', 'Model'], index = stocks)

In [109]:
# initalize result variable
NPExogenousRegressionResults = pd.DataFrame(columns = ['StockName','MAE','MAPE', 'RMSE', 'AdjustedR2', 'Noise'], index = stocks)
for s in stocks:
    print(s)
    try:
        m, forecast , train_metrics, val_metrics = BuildNeuralProphetModelsWithMultipleRegressors(ScriptWiseData[s])
        NPExogenousRegressionResults.loc[s, ['StockName']] =  getStockNameFromStockCode(s)
        NPExogenousRegressionResults.loc[s, ['MAE']] =  val_metrics['MAE'][0]
        NPExogenousRegressionResults.loc[s, ['RMSE']] = np.sqrt(val_metrics['MSE'][0])
        t = forecast.dropna()
        NPExogenousRegressionResults.loc[s, ['MAPE']] = mean_absolute_percentage_error(t.y, t.yhat1)
        NPExogenousRegressionResults.loc[s, ['AdjustedR2']] = r2_score(t.y, t.yhat1)
        NPExogenousRegressionResults.loc[s, ['Noise']] = t.yhat1.diff().abs().mean()
        NPExogenousRegressionResults.loc[s, ['Noise']] = m
        print('Success')
    except:
        print("Error Occured")
        
NPExogenousRegressionResults.to_csv('./Outputs/NeuralProphetRegressionWithExogenousVariablesResults.csv')

BOM524715


  0%|          | 0/288 [00:00<?, ?it/s]

Success
BOM532488


  0%|          | 0/284 [00:00<?, ?it/s]

Success
BOM500124


...

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/282 [00:00<?, ?it/s]

Success
BOM532540
Success
BOM507685


  0%|          | 0/288 [00:00<?, ?it/s]

Success


In [110]:
NPExogenousRegressionResults

,StockName,MAE,MAPE,RMSE,AdjustedR2,Noise
BOM524715,SUN PHARMACEUTICAL INDUSTRIES LTD.,7.687661,0.015471,10.598346,0.979083,<neuralprophet.forecaster.NeuralProphet object...
BOM532488,DIVI'S LABORATORIES LTD.,23.359259,0.018698,29.512802,0.987957,<neuralprophet.forecaster.NeuralProphet object...
BOM500124,DR.REDDY'S LABORATORIES LTD.,45.767691,0.015201,55.161157,0.983793,<neuralprophet.forecaster.NeuralProphet object...
BOM500180,HDFC Bank Ltd,12.291547,0.017491,32.449522,0.987982,<neuralprophet.forecaster.NeuralProphet object...
BOM532174,ICICI BANK LTD.,9.428638,0.045272,11.81925,0.955388,<neuralprophet.forecaster.NeuralProphet object...
BOM500247,KOTAK MAHINDRA BANK LTD.,8.056787,0.01437,12.330192,0.987605,<neuralprophet.forecaster.NeuralProphet object...
BOM500470,TATA STEEL LTD.,6.34917,0.017444,8.682633,0.995097,<neuralprophet.forecaster.NeuralProphet object...
BOM500228,NaN,NaN,NaN,NaN,NaN,NaN
BOM500440,HINDALCO INDUSTRIES LTD.,4.027395,0.032431,5.241284,0.985944,<neuralprophet.forecaster.NeuralProphet object...
BOM532977,BAJAJ AUTO LTD.,22.465128,0.01161,33.031827,0.991249,<neuralprophet.forecaster.NeuralProphet object...


In [ ]:
# # m = NeuralProphet()
# # metrics = m.fit(df, freq="D")
# for s in stocks:
#     print(s)
#     try:
#         m, forecast , train_metrics, val_metrics = BuildNeuralProphetModelsWithMultipleRegressors(ScriptWiseData[s])
# #         m, forecast , train_metrics, val_metrics = BuildNeuralProphetModels(ScriptWiseData[s])
#         print('Succeess')
#     except:
#         print("Error Occured")

In [ ]:
# forecasts_plot = m.plot(forecast)
# fig_comp = m.plot_components(forecast)

In [ ]:
# train_metrics.sort_values(by=['MAE', 'MSE'], ascending = [False, False]).head(3)

In [ ]:
# train_metrics.sort_values(by=['MAE', 'MSE'], ascending = [True, True]).head(3)

In [ ]:
# val_metrics